# Dynamic routing example
[![Dynamic_routing_example.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/Dynamic_routing_example.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/Dynamic_routing_example.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/Dynamic_routing_example.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/Dynamic_routing_example.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/Dynamic_routing_example.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/Dynamic_routing_example.ipynb)](https://colab.ampl.com)

Description: Example of interactive optimization with GUI using AMPL and Google Maps

Tags: amplpy,gui

Notebook author: Christian Valente <<ccv@ampl.com>>

In [13]:
# Install dependencies
%pip install -q amplpy pandas

In [14]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

## Introduction
This example shows how to create a simple GUI for a routing problem that allows to:

* visualize inputs and solutions on a map
* dynamically add points by clicking on the map itself
* show each step of the row generation methodology applied to solve it

### Insert your Google Maps API key
Follow [the instructions](https://jupyter-gmaps.readthedocs.io/en/latest/authentication.html) to generate a Google Maps key for development, then insert it in the placeholder below.

In [15]:
API_KEY = ""  # Fill in with your API key

## Install and import dependencies


In [ ]:
if not API_KEY:
    raise ValueError("Please insert your Google maps API key")
%pip install -q gmaps
import gmaps

gmaps.configure(api_key=API_KEY)
# Activate custom widgets manager
from google.colab import output

output.enable_custom_widget_manager()
# Import display and ipywidgets
from IPython.display import display
import ipywidgets as widgets

# Import needed data/maths libs
import pandas as pd
import math
import collections
import sys

# collections.Iterable after python 3.9
if sys.version_info >= (3, 10):
    collections.Iterable = collections.abc.Iterable

## Google Maps related helper class
This class wraps all the implementation details related to graphically rendering the map and creating the buttons and textboxes that form the graphical interface you'll see at the bottom.

It has to be initialized passing a callback that will be executed when the button *Solve* is clicked.
It also has functions to programmatically add markers to the map, to retrieve the markers location (both the ones added programmatically and the ones added by simply clicking on the map) and to draw lines on the map itself.

In [ ]:
class MyMap:
    def __init__(self, solve_function):
        # Create the google maps widget
        self._figure = gmaps.figure(
            layout={
                "width": "1200px",
                "height": "800px",
                "border": "1px solid black",
                "padding": "1px",
            }
        )
        # Add a marker layer, used to programmatically display markers
        self._markers = gmaps.marker_layer([])
        # Add a drawing layer, used to catch user interactions (additional markers)
        # and to draw the lines on the map
        self._drawing = gmaps.drawing_layer(show_controls=False)
        self._drawing.on_new_feature(self._new_feature_callback)
        self._figure.add_layer(self._drawing)
        # Register a handler to call when the solve button is cliecked
        self._solve_function = solve_function

        # Create a textbox and two buttons
        self._infoBox = widgets.Text(
            description="Status",
            disabled=True,
            layout={"width": "85%", "margin": "10px 0 0 0"},
        )
        self._btn_clear = widgets.Button(
            description="Clean",
            disabled=False,
            button_style="success",
            tooltip="Clear map",
            icon="redo",
        )
        self._btn_solve = widgets.Button(
            description="Solve",
            disabled=False,
            button_style="success",
            tooltip="Solve model",
            icon="bolt",
        )

        self._btn_clear.on_click(self.clear_click)
        self._btn_solve.on_click(self.solve_click)
        self._hcontainer = widgets.HBox([self._btn_clear, self._btn_solve])
        self._container = widgets.VBox([self._figure, self._infoBox, self._hcontainer])

        self.set_solving_status("init")

    def add_points(self, coordinates: list):
        # Programmatically add markers to the map
        markers = [gmaps.Marker(location=c) for c in coordinates]
        self._markers.markers = markers
        self._figure.add_layer(self._markers)

    def get_points(self) -> list:
        # Get the points on the map (both added programmatically and drawn by the user)
        additionalPoints = [
            f for f in self._drawing.features if isinstance(f, gmaps.Marker)
        ]
        return [m.location for m in self._markers.markers + additionalPoints]

    def clean(self, markers=True, lines=True):
        # Clean the map of markers and/or lines
        if markers:
            self._markers.markers = []
            self._drawing.features = [
                f for f in self._drawing.features if not isinstance(f, gmaps.Marker)
            ]
        if lines:
            self._drawing.features = [
                f for f in self._drawing.features if isinstance(f, gmaps.Marker)
            ]

    def solve_click(self, obj):
        # Event handler for button click, execute the callback
        self._solve_function(self, self._status == "solving")

    def _new_feature_callback(self, feature):
        # when a new marker is added to the map, invalidate the previous path,
        # when a line is added, do nothing
        if isinstance(feature, gmaps.Line):
            return
        if self._status != "init":
            self.log("Location added, solution interrupted")
            self.set_solving_status("init")
            self.clean(markers=False)

    def clear_click(self, obj):
        # Event handler for clean button click, clear the map
        self.clean()
        self.log("Reset map")
        self.set_solving_status("init")

    def __draw_route(self, start: tuple, end: tuple):
        # Draw a single line on the map
        return gmaps.Line(start=start, end=end, stroke_weight=4.0, stroke_color="black")

    def add_arcs(self, arcs: list):
        # Add the list of arcs (expressed as a list of tuples of coordinates,
        # e.g. [((startLat1, startLong1), (endLat1, endLong1)),
        #       ((startLat2, startLong2), (endLat2, endLong2)), ...]
        lines = self._drawing.features.copy()
        for start, end in arcs:
            lines.append(self.__draw_route(start, end))
        self._drawing.features = lines

    def set_solving_status(self, status: str):
        # Set the solving status (affects the buttons on the GUI)
        # valid values: init|solving|finished
        self._status = status
        self._btn_solve.description = (
            "Solve" if status != "solving" else "Continue solving"
        )
        self._btn_solve.disabled = status == "finished"

    def log(self, text):
        # set the message box to the specified text
        self._infoBox.value = text

    def render(self):
        # Render the map and its container
        return self._container

## AMPL model: *TSP model with subtour elimination*

The standard integer programming formulation of the Travelling Salesman Problem is as follows:

Given a set of cities $S=\{1,..,n\}$ and distances $c_{ij}$ ($i,j \in S$), we define:

* $x_{ij} \in \{0,1\}  \forall i \in S, j \in S, i \ne j $ a variable denoting if path from $i$ to $j$ or from $j$ to $i$ is part of ours solution.

Our aim is to minimize the total tour length:

* $\sum_{i=1}^{n} \sum_{j=1,j \neq i}^{n} c_{ij}x{ij}$ 

while visiting all cities once, which is enforced with these two constraints:

* $\sum_{i=1,i \neq j} x_{ij} \forall j \in S$

and

* $\sum_{j=1,j \neq i} x_{ij} \forall i \in S$

This formulation notably allows for subtours that can be eliminated with a suitable *subtour-elimination* constraint; the number of possible subtours grows exponentially with the number of cities, so one approach is to eliminate only the subtours that result from actually obtained solutions.

The following AMPL model expresses the problem above with a slight semplification: we suppose the distance/cost matrix is symmetrical (aka the distances between each pair of cities is the same both ways), so we can consider only half the possible pairs for $X$. So we define:

*  $x_{ij} \in \{0,1\}  \forall i \in S, j \in S, i > j $

and we change the formulation accordingly.

In [ ]:
%%ampl_eval
# List of nodes
set NODES ordered;
# Longitude and latitude of each node; not used in the model,
# just for data retrieval
param longitude {NODES};
param latitude {NODES};
# Set of valid city pairs (segments)
set PAIRS := {i in NODES, j in NODES: ord(i) < ord(j)};
# Distance of each valid route
param distance {(i,j) in PAIRS};

# If set to 1, the relative segment is part of the solution
var X{PAIRS} binary;
# Minimize the total tour length
minimize Tour_Length: sum {(i,j) in PAIRS} distance[i,j] * X[i,j];

# Each node must have two segments active, showing that we get in
# and out of it
subject to Visit_All {i in NODES}:
   sum {(i, j) in PAIRS} X[i,j] + sum {(j, i) in PAIRS} X[j,i] = 2;

# The following are used in the subtour elimination procedure
param nSubtours >= 0 integer, default 0;
set SUB {1..nSubtours} within NODES;
# Subtour elimination constraint
# For each stored subtour, make sure that there is at least one segment
# leading into the subtour itself (so that it is not a subtour any longer)
subject to Subtour_Elimination {k in 1..nSubtours}:
  sum {i in SUB[k], j in NODES diff SUB[k]} 
      if (i, j) in PAIRS then X[i, j] else X[j, i] >= 2;

### Model-related functions
For convenience in getting data from the underlying `AMPL` object, we addd the following methods to the class itself. 
These can then be called via normal members function syntax, e.g. `ampl.function_name()`.

In [ ]:
def get_arcs(self) -> list:
    return [
        (i, j)
        for (i, j, k) in self.get_data(
            "{(i,j) in PAIRS : X[i,j] > 0} X[i,j];"
        ).to_list()
    ]


AMPL.get_arcs = get_arcs


def get_coords(self) -> dict:
    return self.get_data("latitude, longitude").to_dict()


AMPL.get_coords = get_coords


def get_nodes(self) -> set:
    return set(self.get_set("NODES").members())


AMPL.get_nodes = get_nodes

### Subtour-related helper functions
These functions identify subtours from a list of arcs. Note that we could have also dynamically added to the AMPL object, we have chosen not-to to show both approaches.

In [ ]:
import math
from itertools import tee


# Graphs helper routines
def trasverse(node, arcs: list, allnodes: set, subtour=None) -> list:
    # Trasverses all the arcs in the set arcs, starting from node
    # and returns the tour
    if not subtour:
        subtour = list()
    # Find arcs involving the current node
    myarcs = [(i, j) for (i, j) in arcs if node == i or node == j]
    if len(myarcs) == 0:
        return
    # Append the current node to the current subtour
    subtour.append(node)
    # Use the first arc found
    myarc = myarcs[0]
    # Find destination (or origin) node
    destination = next(i for i in myarc if i != node)
    # Remove from arcs and nodes to visit
    arcs.remove(myarc)
    if node in allnodes:
        allnodes.remove(node)

    trasverse(destination, arcs, allnodes, subtour)
    return subtour


def as_steps(iterable):
    a, b = tee(iterable)
    next(b, None)
    return list(zip(a, b))


def find_subtours(arcs: list, allnodes: set):
    # Find all the subtours defined by a set of arcs and
    # return them as a list of list
    subtours = list()
    allnodes = allnodes.copy()
    while len(allnodes) > 0:
        l = trasverse(next(iter(allnodes)), arcs, allnodes)
        subtours.append(l)
    return subtours

## Data related helpers
Set of functions designed to provide sample data and preprocess it so that it has all the properties needed for the solution process.

In [ ]:
def haversine(coord1, coord2):
    # Calculate distance between coordinates on Earth's surface
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = (
        math.sin(dphi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    )

    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))


def define_data(coordinates: list, names: list = None) -> tuple[list, pd.DataFrame]:
    # Get two dataframes (one with the properly coordinates, one with the distances) given a list of
    # coordinates
    df = pd.DataFrame.from_records(
        coordinates, columns=["latitude", "longitude"], index=names
    )
    df.index.name = "NODE"
    nodes = list(df.index)
    distances = {
        (orig, dest): [
            round(
                haversine(
                    (df.loc[orig]["latitude"], df.loc[orig]["longitude"]),
                    (df.loc[dest]["latitude"], df.loc[dest]["longitude"]),
                )
                / 1000
            )
        ]
        for orig in nodes
        for dest in nodes
        if nodes.index(dest) > nodes.index(orig)
    }
    return (df, distances)


def sample_data():
    # Get some sample data
    otherLocs = [
        {
            "name": "Bolzano Airport",
            "city": "Bolzano",
            "country": "Italy",
            "iata": "BZO",
            "latitude": 46.4626,
            "longitude": 11.3264,
        },
        {
            "name": "Ottawa Macdonald-Cartier International Airport",
            "city": "Ottawa",
            "country": "Canada",
            "iata": "YOW",
            "latitude": 45.3208,
            "longitude": -75.6724,
        },
        {
            "name": "Porto Airport",
            "city": "Porto",
            "country": "Portugal",
            "iata": "OPO",
            "latitude": 41.2481,
            "longitude": -8.681389,
        },
        {
            "name": "Newark Liberty International Airport",
            "city": "Newark",
            "country": "United States",
            "iata": "EWR",
            "latitude": 40.6925,
            "longitude": -74.168611,
        },
        {
            "name": "Madrid Barajas Airport",
            "city": "Madrid",
            "country": "Spain",
            "iata": "MAD",
            "latitude": 40.493556,
            "longitude": -3.566764,
        },
        {
            "name": "Chicago O'Hare International Airport",
            "city": "Chicago",
            "country": "United States",
            "iata": "ORD",
            "latitude": 41.978611,
            "longitude": -87.904722,
        },
        {
            "name": "Albuquerque International Sunport",
            "city": "Albuquerque",
            "country": "United States",
            "iata": "ABQ",
            "latitude": 35.0401,
            "longitude": -106.6094,
        },
        {
            "name": "Melbourne Airport",
            "city": "Melbourne",
            "country": "Australia",
            "iata": "MEL",
            "latitude": -37.673333,
            "longitude": 144.843333,
        },
        {
            "name": "San Francisco International Airport",
            "city": "San Francisco",
            "country": "United States",
            "iata": "SFO",
            "latitude": 37.618889,
            "longitude": -122.375,
        },
    ]
    (coords, distances) = define_data(
        [(c["latitude"], c["longitude"]) for c in otherLocs],
        [c["city"] for c in otherLocs],
    )
    return (coords, distances)

# AMPL solution method
To solve the TSP model, we need to make sure it has no subtours; since enumerating them all beforehand is basically impossible, we choose a reactive approach. This function solves the model (with the current set of subtours cut out) and checks if the obtained solution has any subtour.
If it does, it displays it and return False, otherwise the solution is a valid path, simply return True.


In [ ]:
def solve_ampl(map) -> bool:
    nSubtoursParam = ampl.get_parameter("nSubtours")
    SubtoursSet = ampl.get_set("SUB")
    allsubtours = [
        ampl.get_set("SUB")[i].to_list() for i in range(1, int(nSubtoursParam.value()))
    ]

    # Solve the model
    ampl.solve()

    # Get solution
    print("Solved, getting solution")
    length = ampl.get_objective("Tour_Length").value()
    print(ampl.get_nodes())
    print(ampl.get_arcs())

    subtours = find_subtours(ampl.get_arcs(), ampl.get_nodes())
    # If we have only one tour, the solution is valid
    if len(subtours) <= 1:
        print(f"\nFound only {len(subtours)} subtours, returning")
        map.log(f"Length={length}. No subtour found, shortest valid path.")
        return True

    # Else:
    print(f"\nFound {len(subtours)} subtours, plotting them and adding cuts")
    map.log(f"Length={length}. {len(subtours)} subtours, adding cuts")

    # Plot the subtours
    coords = ampl.get_coords()
    for s in subtours:
        vt = s.copy()
        vt.append(vt[0])
        vt = as_steps(vt)
        vt = [(coords[i], coords[j]) for i, j in vt]
        map.add_arcs(vt)

    # Add the current tours to the list of subtours
    allsubtours.extend(subtours)
    # And add those to the constraints by assigning the values to
    # the parameter and the set
    nSubtoursParam.set(len(allsubtours))
    for i, tour in enumerate(allsubtours):
        print(f"Adding cut: {tour}")
        SubtoursSet[i + 1].set_values(tour)
    return False

# "Glue" functions
These functions help glueing the GUI and python data processing part with the underlying AMPL object.


In [ ]:
SOLVER = "highs"
from amplpy import DataFrame as aDF


def data_2_AMPL(coordinates: pd.DataFrame, distances: pd.DataFrame):
    # Convert the dataframes to the specific dataframes expected by AMPL and assings
    # those values to the set of NODES, the params longitude and latitude, and the parameter distances
    ampl.eval("reset data;")
    # Convert to amplpy dataframes
    coordinates = aDF.from_pandas(coordinates, index_names=["POINT"])
    distances = aDF.from_dict(
        distances, index_names=["orig", "dest"], column_names=["distance"]
    )
    ampl.set_data(coordinates, set_name="NODES")
    ampl.set_data(distances)


def solve(map: MyMap, alreadySolving: bool):
    # Main solve procedure; if we're not in the middle of a solution process,
    # reads the data from the map and pass it to AMPL then solve it.
    # Otherwise just continue the ongoing solution process
    if not alreadySolving:
        map.set_solving_status("solving")
        nodes = map.get_points()
        if len(nodes) <= 2:
            map.log("Trivial problem, add more points to solve.")
            map.set_solving_status("finished")
            return
        print(f"Solving for {len(nodes)} points")
        (coords, distances) = define_data(nodes)
        data_2_AMPL(coords, distances)
        ampl.option["solver"] = SOLVER
    continue_solve(map)


def continue_solve(map: MyMap):
    # Continue the ongoing solution process
    # Clear the previous solution from the map
    map.clean(markers=False)
    # Call AMPL to solve the current problem instance
    status = solve_ampl(map)
    # Status==True if the solution has no subtours
    if status:
        # If the solution is valid (no subtours), "inform" the map object and
        # display the solution
        map.set_solving_status("finished")
        arcs = ampl.get_arcs()
        coords = ampl.get_coords()
        tours = [(coords[i], coords[j]) for i, j in arcs]
        print(f"Solution has {len(tours)} arcs;")
        print(arcs)
        map.add_arcs(tours)

## Show the UI
These commands load a sample dataset and add it to the map

In [ ]:
# Create the GUI and use the solve function defined here to actually solve
# the model
m = MyMap(solve)
# Get sample data
(coord, dist) = sample_data()
# Add it to the map
m.add_points(coord[["latitude", "longitude"]].to_records(index=False))
# Show the GUI
m.render()